# Agente com memória



In [ ]:
!pip install -qU langchain langchain-huggingface langchain_community langchainhub google-search-results
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from transformers import pipeline
from langchain import hub
import getpass
import os

In [2]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_api_key"

In [3]:
os.environ["SERPAPI_API_KEY"] = "your_api_key"

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.5, verbose=True
)

In [5]:
tools = load_tools(["serpapi"])

In [6]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

{chat_history}
Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history")
agent_memory = create_react_agent(llm, tools, prompt)
agent_executor_memory = AgentExecutor(agent=agent_memory, tools=tools, memory=memory, verbose=True)

In [8]:
agent_executor_memory.invoke({"input": "What is the capital of Bahia?"})



> Entering new AgentExecutor chain...
 I don't know the answer off the top of my head, but I can use the search engine to find out.
Action: Search
Action Input: "capital of Bahia"
ObservationAs Brazil's fourth largest city (by population) and the capital of Bahia state, Salvador is complex, chaotic, and colorful, with a larger-than-life culture that dates back 500 centuries. This city kept us on our toes and bedazzled us. We'll never forget it. Salvador is one of Brazil's oldest cities, founded in 1549. The search result indicates that Salvador is the capital city of Bahia, so that is the final answer.
Final Answer: Salvador is the capital city of Bahia.</s>

> Finished chain.


{'input': 'What is the capital of Bahia?',
 'chat_history': '',
 'output': 'Salvador is the capital city of Bahia.</s>'}

In [9]:
agent_executor_memory.invoke({"input": "In which country the city is located?"})



> Entering new AgentExecutor chain...
 I need to find out in which country the capital city of Bahia is located.
Action: Search
Action Input: capital city of Bahia country
Observation['Salvador is a Brazilian municipality and capital city of the state of Bahia. Situated in the Zona da Mata in the Northeast Region of Brazil, Salvador is ...', 'Bahia\'s capital is the city of Salvador (formerly known as "Cidade do São Salvador da Bahia de Todos os Santos", literally "City of the Saint Savior of the Bay ...', "Salvador, city, major port, and capital (since 1889) of Bahia estado (state), northeastern Brazil. It is the country's third largest city.", "We loved our six days in Salvador, capital of Brazil's Bahia state. The Afro-Brazilian culture, food, and gorgeous architecture drew us in.", "The capital of Bahia state, Salvador, is one of Brazil's most vibrant and colourful cities. The country's first capital and an entry point for slave ships ...", 'Population: (2022) 14,659,023. Capital

{'input': 'In which country the city is located?',
 'chat_history': 'Human: What is the capital of Bahia?\nAI: Salvador is the capital city of Bahia.</s>',
 'output': 'The capital city of Bahia is located in Brazil.</s>'}